0.0 Imports and setup

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import tensorflow_datasets as tfds
from keras import Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, Dropout, MaxPooling2D, Dense, Flatten


import montecarlo_dropout


Dataset imagenet for validation crossplatform (not needed at the moment but maybe helpfull later)

In [12]:
(ds_test), ds_info = tfds.load(
    'imagenet_v2',
    split=['test[:90%]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /home/emilhillebrand/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0.incompleteSBAT6E/imagen…

Dataset imagenet_v2 downloaded and prepared to /home/emilhillebrand/tensorflow_datasets/imagenet_v2/matched-frequency/3.0.0. Subsequent calls will reuse this data.


2023-12-14 16:55:47.971231: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Load Image

In [24]:
testtabby = load_img("./Pictures/tabby-cat.jpeg")
testtabby = testtabby.resize((299, 299))
#ytest = img_to_array(y_test)
testtabby = img_to_array(testtabby)
testtabby_input = np.expand_dims(testtabby, axis=0)
testtabby = keras.applications.xception.preprocess_input(testtabby_input[None])  

Normalize function (not needed at the moment)

In [2]:


def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(299)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)



NameError: name 'ds_test' is not defined

Model no Monte Carlo

In [4]:
modelNoMC = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

modelNoMC.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]

Model Monte Carlo Dropout

In [25]:
#From https://nchlis.github.io/2017_08_10/page.html 

nfilters = [64,128,256]
ndense = 512
add_BatchNorm = True
dropout_rate_conv = [0.0, 0.0, 0.0]#1 value for each conv block
dropout_rate_dense = 0.0

model_id='CNN_bn_'+str(add_BatchNorm)+'_dropConv_'+str(dropout_rate_conv[0])+'_'+\
str(dropout_rate_conv[1])+'_'+str(dropout_rate_conv[2])+'_'+'dropDense_'+str(dropout_rate_dense)
print('Build model...',model_id)

model = Sequential()

#Conv block #1
model.add(Conv2D(nfilters[0], (3, 3), padding='same',
                 input_shape=testtabby_input.shape[1:]))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_conv[0]))

model.add(Conv2D(nfilters[0], (3, 3)))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_conv[0]))

model.add(MaxPooling2D(pool_size=(2, 2)))

#Conv block #2
model.add(Conv2D(nfilters[1], (3, 3), padding='same'))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_conv[1]))

model.add(Conv2D(nfilters[1], (3, 3)))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_conv[1]))

model.add(MaxPooling2D(pool_size=(2, 2)))

#Conv block #3
model.add(Conv2D(nfilters[2], (3, 3), padding='same'))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_conv[2]))

model.add(Conv2D(nfilters[2], (3, 3)))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_conv[2]))

model.add(MaxPooling2D(pool_size=(2, 2)))

#at this point each image has shape (None, 2, 2, nfilters[2])
model.add(Flatten())
#at this point each image has shape (None, 2*2*nfilters[2])

model.add(Dense(ndense))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_dense))

model.add(Dense(ndense))
if(add_BatchNorm==True):
    model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(dropout_rate_dense))

model.add(Dense(1000))
model.add(Activation('softmax'))

model.summary()

Build model... CNN_bn_True_dropConv_0.0_0.0_0.0_dropDense_0.0
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 299, 299, 64)      1792      
                                                                 
 batch_normalization_36 (Ba  (None, 299, 299, 64)      256       
 tchNormalization)                                               
                                                                 
 activation_33 (Activation)  (None, 299, 299, 64)      0         
                                                                 
 dropout_31 (Dropout)        (None, 299, 299, 64)      0         
                                                                 
 conv2d_32 (Conv2D)          (None, 297, 297, 64)      36928     
                                                                 
 batch_normalization_37 (Ba  (None, 297, 297, 64)      25

ModelMC Converter Test extern (not working) from https://github.com/statsu1990/monte_carlo_dropout

In [ ]:
model = tf.keras.applications.MobileNetV2()

model.save("./model.keras")

modelMC = montecarlo_dropout.MontecarloDropout()
modelMC.build_model("./model.keras")

#modelMC = build_model(modelNoMC)
modelMC.summary()

Make a MC D model my Own not working :(

In [7]:

layers = []
dropout_rate = 0.4
#model = tf.keras.models.Sequential()
#for i in range(len(modelNoMC.layers)):
#    model.add(modelNoMC.layers.pop())
#    model.add(tf.keras.layers.Dropout(dropout_rate))
    #print(i)
#    layers.append(modelNoMC.layers[i])
    
#print((layers))


#layers.reverse

input_layer = modelNoMC.layers[0]
x = modelNoMC.layers[1](input_layer)

for i in range(len(modelNoMC.layers)-2):
    x = modelNoMC.layers[i+2](x)
    x = tf.keras.layers.Dropout(dropout_rate, training=True)(x)
    if(i == (len(layers)-2)):
        output_layer = modelNoMC.layers[i+2](x)

modelMC = keras.model.Model(inputs=input_layer, outputs=output_layer)

#modelMC.build((None, 299,299, 3))
#predictions = modelMC.predict(preprocess)
modelMC.summary()

TypeError: Inputs to a layer should be tensors. Got '<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7f067a1d3010>' (of type <class 'keras.src.layers.convolutional.conv2d.Conv2D'>) as input for layer 'block1_conv1_bn'.

Weight adjustment due to dropout (later)

In [121]:
WEIGHTS = modelNoMC.get_weights()
WEIGHTS[0] *= 1/0.4
modelNoMC.set_weights(WEIGHTS)

models Compile

In [26]:
modelNoMC.compile(
optimizer='Adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

model.compile(
optimizer='Adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)

predict

In [70]:
predictions = modelNoMC.predict(testtabby_input)
predictions = keras.applications.xception.decode_predictions(predictions)
print(predictions)

print("With MC")

predictions = model.predict(testtabby_input)
predictions = keras.applications.xception.decode_predictions(predictions)
print(predictions)

1/1 [==============================] - 1s 525ms/step
[[('n02123045', 'tabby', 0.6505307), ('n02123159', 'tiger_cat', 0.23030522), ('n02124075', 'Egyptian_cat', 0.01496593), ('n02127052', 'lynx', 0.0042105187), ('n02123394', 'Persian_cat', 0.0010178831)]]
With MC
1/1 [==============================] - 1s 600ms/step
[[('n03788195', 'mosque', 0.3829502), ('n02102177', 'Welsh_springer_spaniel', 0.21382879), ('n07760859', 'custard_apple', 0.199601), ('n02643566', 'lionfish', 0.040480074), ('n03085013', 'computer_keyboard', 0.03388513)]]


MonteCarlo predict

In [69]:
y_samples = []
for i in range(100):
    predictions = model.predict(testtabby_input)
    y_samples.append(keras.applications.xception.decode_predictions(predictions))
    #print(y_samples[i])

1/1 [==============================] - 1s 594ms/step


Output MC D

In [72]:
y_samples = np.stack([model(testtabby_input,training=True)
                   for sample in range(100)])



In [73]:
y_samples_readable = []
for i in range(len(y_samples)):
    y_samples_readable.append(keras.applications.xception.decode_predictions(y_samples[i])) 

print(y_samples_readable)

[[[('n15075141', 'toilet_tissue', 0.001), ('n02319095', 'sea_urchin', 0.001), ('n02395406', 'hog', 0.001), ('n02391049', 'zebra', 0.001), ('n02389026', 'sorrel', 0.001)]], [[('n15075141', 'toilet_tissue', 0.001), ('n02319095', 'sea_urchin', 0.001), ('n02395406', 'hog', 0.001), ('n02391049', 'zebra', 0.001), ('n02389026', 'sorrel', 0.001)]], [[('n15075141', 'toilet_tissue', 0.001), ('n02319095', 'sea_urchin', 0.001), ('n02395406', 'hog', 0.001), ('n02391049', 'zebra', 0.001), ('n02389026', 'sorrel', 0.001)]], [[('n15075141', 'toilet_tissue', 0.001), ('n02319095', 'sea_urchin', 0.001), ('n02395406', 'hog', 0.001), ('n02391049', 'zebra', 0.001), ('n02389026', 'sorrel', 0.001)]], [[('n15075141', 'toilet_tissue', 0.001), ('n02319095', 'sea_urchin', 0.001), ('n02395406', 'hog', 0.001), ('n02391049', 'zebra', 0.001), ('n02389026', 'sorrel', 0.001)]], [[('n15075141', 'toilet_tissue', 0.001), ('n02319095', 'sea_urchin', 0.001), ('n02395406', 'hog', 0.001), ('n02391049', 'zebra', 0.001), ('n0238

In [74]:
counter = 0
counterCount = 0
for i in range(len(y_samples_readable)):
    for x in range(len(y_samples_readable[0][0])):
        print(y_samples_readable[i][0][x][1])
        counterCount = counterCount +1
        if(y_samples_readable[i][0][x][1] == "sorrel"):
            counter = counter +1
y_samples_mean = counter / counterCount
print(y_samples_mean)

toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra
sorrel
toilet_tissue
sea_urchin
hog
zebra